In [1]:
%config IPCompleter.greedy=True

In [66]:
import pandas as pd
import numpy as np
from pathlib import Path

In [16]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"

In [17]:
!ls $xcelLocation | grep ".xlsx" > xlFiles
tmp = !cat xlFiles
names = [name[:-5] for name in tmp]

In [31]:
def convert(xlFileName, returnAllMode=False):
    try:
        if (not Path(xcelLocation + xlFileName + '.csv').is_file()) or returnAllMode:
            xl = pd.read_excel(xcelLocation + xlFileName + ".xlsx", index_col=0,
                               header=[0], skiprows=[0,1])
            xl.to_csv(xcelLocation + xlFileName + '.csv', encoding='utf-8', index=False)
    except:
        xl = None
        print(xlFileName)
    finally:
        return xl

In [40]:
allDFs = []
allDFNames = []
for name in names:
    tmp = convert(name, returnAllMode=True)
    if tmp is not None:
        allDFs.append(tmp.copy())
        allDFNames.append(name)

1380-10-2
1380-1-11
1380-11-29
1380-1-14
1380-1-18
1380-1-19
1380-1-20
1380-12-14
1380-12-19
1380-1-21
1380-12-20
1380-1-22
1380-1-25
1380-12-5
1380-1-26
1380-1-27
1380-1-28
1380-1-29
1380-1-5
1380-1-6
1380-1-7
1380-1-8
1380-2-11
1380-2-12
1380-2-17
1380-2-18
1380-2-19
1380-2-22
1380-2-23
1380-2-25
1380-2-26
1380-2-30
1380-2-31
1380-2-3
1380-2-4
1380-2-5
1380-3-22
1380-3-23
1380-3-29
1380-3-5
1380-3-6
1380-3-9
1380-4-19
1380-4-2
1380-6-11
1380-8-15
1380-8-28
1380-9-10
1380-9-14
1380-9-18
1380-9-7
1381-10-4
1381-1-10
1381-11-19
1381-11-1
1381-11-28
1381-11-30
1381-11-5
1381-12-11
1381-12-19
1381-12-6
1381-2-14
1381-2-18
1381-3-21
1381-3-25
1381-3-5
1381-4-10
1381-4-23
1381-4-31
1381-5-22
1381-6-11
1381-6-17
1381-6-31
1381-6-9
1381-7-24
1381-8-11
1381-8-15
1381-8-22
1381-9-19
1381-9-24
1381-9-30
1382-10-24
1382-11-19
1382-12-17
1382-12-20
1382-12-26
1382-12-27
1382-12-2
1382-12-5
1382-2-20
1382-3-12
1382-3-17
1382-3-28
1382-3-5
1382-4-24
1382-4-25
1382-4-8
1382-5-4
1382-6-22
1382-6-24
13

In [67]:
print(len(allDFs))
def makeMasterTable():
    for index, df in enumerate(allDFs):
        date = np.full(len(df), allDFNames[index]).tolist()
        df["date"] = date
    xl = pd.concat(allDFs, keys=allDFNames, ignore_index=True)
    xl.to_csv(xcelLocation + 'master.csv', encoding='utf-8', index=False)
    return xl
allDF = makeMasterTable()

2186


In [69]:
allDF

,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,date
0,ايران‌ خودرو,702,1606486,5528171211,3450,3420,3420,-30,-0.87,3420,-30,-0.87,3350,3480,1380-10-10
1,صنايع‌مخابراتي‌راه‌دورايران‌,194,224015,4437404551,19800,19836,19840,40,0.20,19840,40,0.20,19800,19840,1380-10-10
2,گروه س توسعه صنعتي ايران,159,393671,393671000,1000,1000,1000,0,0.00,1000,0,0.00,1000,1000,1380-10-10
3,توسعه‌معادن‌وفلزات‌,139,901134,996606338,1055,1107,1098,43,4.08,1098,43,4.08,1059,1107,1380-10-10
4,توليدي‌ كاشي‌ تكسرام‌,127,245425,1950745485,7871,7949,7949,78,0.99,7949,78,0.99,7794,7949,1380-10-10
5,سرمايه گذاري گروه توسعه ملي,106,474629,2346271212,4950,4950,4950,0,0.00,4950,0,0.00,4920,4950,1380-10-10
6,چيني‌ ايران,99,243691,3436651610,14070,14210,14080,10,0.07,14080,10,0.07,13930,14210,1380-10-10
7,سرمايه‌گذاري‌غدير(هلدينگ‌,92,173747,507257982,2904,2920,2901,-3,-0.10,2901,-3,-0.10,2900,2930,1380-10-10
8,سرمايه گذاري توسعه صنعت وتجارت,85,170057,475976404,2790,2789,2800,10,0.36,2800,10,0.36,2789,2800,1380-10-10
9,توسعه‌ معادن‌ روي‌ ايران‌,83,176780,381755908,2160,2149,2195,35,1.62,2195,35,1.62,2148,2195,1380-10-10
